In [3]:
# Import libraries
import pandas as pd
import json
from pymongo import MongoClient
import mysql.connector
import time
from sqlalchemy import create_engine,text
from datetime import date

In [ ]:
df_answers = pd.read_csv("dataset/Answers.csv", encoding="ISO-8859-1")
df_questions = pd.read_csv("dataset/Questions.csv", encoding="ISO-8859-1")
df_tags = pd.read_csv("dataset/Tags.csv", encoding="ISO-8859-1")

In [ ]:
df_questions['CreationDate'] = pd.to_datetime(df_questions['CreationDate'])
df_answers['CreationDate'] = pd.to_datetime(df_answers['CreationDate'])

In [ ]:
#limpeza
df_questions = df_questions.dropna()
df_questions['OwnerUserId'] = df_questions['OwnerUserId'].apply(int)
df_questions['Title'] = df_questions['Title'].astype('string')
df_questions['Body'] = df_questions['Body'].astype('string')

In [ ]:
df_questions['year'] = df_questions['CreationDate'].dt.year
df_questions['month'] = df_questions['CreationDate'].dt.month
df_questions['day'] = df_questions['CreationDate'].dt.day
df_questions['hour'] = df_questions['CreationDate'].dt.hour
df_questions['minute'] = df_questions['CreationDate'].dt.minute
df_questions['second'] = df_questions['CreationDate'].dt.second

df_questions = df_questions.drop(columns=['CreationDate'])

df_questions['Date1'] = df_questions[df_questions.columns[5:8]].apply(
     lambda x: '/'.join(x.dropna().astype(str)),
     axis=1
)

df_questions['Date2'] = df_questions[df_questions.columns[8:11]].apply(
     lambda x: ':'.join(x.dropna().astype(str)),
     axis=1
)

df_questions['CreationDate'] = df_questions[df_questions.columns[11:13]].apply(
     lambda x: ' '.join(x.dropna().astype(str)),
     axis=1
)

df_questions = df_questions.drop(columns=['year', 'month', 'day', 'hour', 'minute', 'second', 'Date1', 'Date2'])

In [ ]:
# Setting MongoDB database

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017")

# Drop and create database
client.drop_database("stackoverflow")
db = client['stackoverflow']

# Drop and create collections
db['Questions'].drop()
db['Answers'].drop()
db['Tags'].drop()
collection_questions = db['Questions']
collection_answers = db['Answers']
collection_tags = db['Tags']

# Function to insert data into MongoDB
def insert_into_mongo(df, collection):
    initial_count = collection.count_documents({})
    print(f"# documents: {initial_count}") 
    try:
        data_dict = df.to_dict(orient="records")
        collection.insert_many(data_dict)
        valid_count = collection.count_documents({}) - initial_count
        print(f"{valid_count} documents inserted.")
    except Exception as e:
        attempted_count = len(data_dict)
        valid_count = collection.count_documents({}) - initial_count
        print(f"{valid_count} documents inserted")
        print(f"{attempted_count - valid_count} documents failed to insert")

insert_into_mongo(df_questions, collection_questions)
insert_into_mongo(df_answers, collection_answers)
insert_into_mongo(df_tags, collection_tags)

NameError: name 'df_questions' is not defined

In [ ]:
# Setting MySQL Database
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password = '!Leandro02#',
    database = "stackoverflow"
)
print(mydb)

mycursor = mydb.cursor()

#Creating Table
mycursor.execute("CREATE TABLE questions (id INT AUTO_INCREMENT PRIMARY KEY, OwnerUserId INT, CreationDate VARCHAR(255), score INT, title VARCHAR(255), body VARCHAR(255))")
mycursor.execute("CREATE TABLE answers (id INT AUTO_INCREMENT PRIMARY KEY, OwnerUserId INT, CreationDate VARCHAR(255), parentid INT, score INT, body VARCHAR(255)) foreign key(parentid) references questions(id)")
mycursor.execute("CREATE TABLE tags (tag VARCHAR(255) PRIMARY KEY)")
mycursor.execute("CREATE TABLE taglink (id INT, tag VARCHAR(255)) foreign key(id) references questions(id) foreign key(tag) references tags(tag)")

# Check if Table Exists
mycursor.execute("SHOW TABLES")
for x in mycursor:
    print(x)

In [ ]:
queries = [
    {"description": "Retrieve all questions that have a score greater than 300", "query": ""},
    
    {"description": "Retrieve all answers from 2008", "query": ""},
    
    {"description": "Retrive all questions that have 2 tags", "query": ""},

    {"description": "Retrive all questions from 2015 about PDF that have more than 4 answers", "query": ""}

]

In [ ]:
mongo_times = []
for q in queries:
    start_time = time.time()
    
    if "score greater" in q["description"]:
        query = {'score' : {'$gt' : 300}}
        result = collection_questions.find(query)

    elif "answers from 2008" in q["description"]:
        query = {'creationdate': {'$regex' : '^2008-.*'}}
        result = collection_answers.find(query)

    elif "2 tags" in q["description"]:
        query = {}
        result = 
    else:
        query = {}
        result = 
        
    end_time = time.time()
 
    list_result = list(result)
    mongo_times.append(end_time - start_time)
    print(f"{q['description']}: {len(list_result)} found in {end_time - start_time:.4f} seconds")
client.close()

In [ ]:
mysql_times = []
with engine.connect() as connection:
    for q in queries:
        start_time = time.time()
        if "score greater" in q["description"]:
            result = connection.execute(text("SELECT title FROM questions WHERE score > 300"))

        elif "answers from 2008" in q["description"]:
            result = connection.execute(text("SELECT * FROM answers WHERE creationdate LIKE '2008%'"))
        
        elif "2 tags" in q["description"]:
            result = connection.execute(text("(SELECT id FROM tags GROUP BY id HAVING COUNT(tag) = 2) AS t JOIN questions q ON t.id = q.id"))
            
        else:
            result = connection.execute(text("SELECT title FROM questions WHERE creationdate LIKE '2015%' AND title LIKE '%PDF%' AND (SELECT COUNT(*) FROM answers WHERE parentid = questions.id) > 4")) #maybe we need a join to compare parentid to questions.id
        results =  result.fetchall()
        end_time = time.time()
        mysql_times.append(end_time - start_time)
        print(f"{q['description']}: {len(results)} results found in {end_time - start_time:.4f} seconds")

In [ ]:
comparison_df = pd.DataFrame({
    "Query": [q["description"] for q in queries],
    "MongoDB_Time(s)": mongo_times,
    "MySQL_Time(s)": mysql_times
})

print(comparison_df)